In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [7]:
import os
folder_path = '/content/drive/MyDrive/Projects/ULX3s-Superresolution-CNN'
os.chdir(folder_path)

In [8]:
!git init
!git status

Reinitialized existing Git repository in /content/drive/MyDrive/Projects/ULX3s-Superresolution-CNN/.git/
Refresh index: 100% (36/36), done.
On branch main
Your branch is up to date with 'origin/main'.

Changes not staged for commit:
  (use "git add <file>..." to update what will be committed)
  (use "git restore <file>..." to discard changes in working directory)
  (commit or discard the untracked or modified content in submodules)
	modified:   Efficient-Teacher-Student-Distillative-Super-Resolution-Model (new commits, untracked content)

Untracked files:
  (use "git add <file>..." to include in what will be committed)
	Pytorch_model_weights.ipynb
	formatted_weights.txt

no changes added to commit (use "git add" and/or "git commit -a")


In [10]:
!git add .
!git commit -m "Added Pytorch to weights File"
!git push

Author identity unknown

*** Please tell me who you are.

Run

  git config --global user.email "you@example.com"
  git config --global user.name "Your Name"

to set your account's default identity.
Omit --global to set the identity only in this repository.

fatal: unable to auto-detect email address (got 'root@1c78a7346ff5.(none)')
fatal: could not read Username for 'https://github.com': No such device or address


In [ ]:
import torch
import numpy as np

class StudentModel(torch.nn.Module):
    def __init__(self, scale_factor=4):
        super(StudentModel, self).__init__()
        self.upsample = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3 * (scale_factor ** 2), kernel_size=3, stride=1, padding=1),
            torch.nn.PixelShuffle(scale_factor)
        )
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv5 = torch.nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.relu(x)
        return x

# Load your model with weights
model = StudentModel()  # Make sure to define your actual model class
model.load_state_dict(torch.load('/content/drive/MyDrive/Projects/ULX3s-Superresolution-CNN/Efficient-Teacher-Student-Distillative-Super-Resolution-Model/ESRGAN/models/SupervisedLossOnFeatureMaps_v2.pth'))  # Load the trained model weights

def extract_and_format_weights(model):
    frac_bits = 4  # Number of fractional bits
    scale_factor = 2 ** frac_bits  # Scaling factor based on the fractional bits

    formatted_weights = ""
    for name, param in model.named_parameters():
        if 'conv' in name and 'weight' in name:
            # Replace periods with underscores for Verilog identifiers
            verilog_name = name.replace('.', '_')
            weights = param.data.cpu().numpy()
            weights_scaled = np.round(weights * scale_factor).astype(np.int32)
            max_weight = np.max(np.abs(weights_scaled))
            num_bits = int(max_weight).bit_length() + 1  # +1 for sign bit

            #print(weights)

            num_kernels, kernel_depth, kernel_height, kernel_width = weights.shape
            # print(weights.shape)
            # Print layer configuration details
            print(f"Layer: {verilog_name}")
            print(f"  Number of Kernels (Output Channels): {num_kernels}")
            print(f"  Kernel Height: {kernel_height}, Kernel Width: {kernel_width}, Kernel Depth: {kernel_depth}\n")

            formatted_weights += f"// Weights for {verilog_name}, each with {num_bits} bits\n"
            for k in range(num_kernels):
                formatted_weights += f"reg signed [{num_bits-1}:0] {verilog_name}_kernel{k}[0:{kernel_depth-1}][0:{kernel_height-1}][0:{kernel_width-1}] = '{{\n"
                for d in range(kernel_depth):
                    for i in range(kernel_height):
                        formatted_weights += "{"
                        for j in range(kernel_width):
                            int_value = int(weights[k, d, i, j] * scale_factor)
                            formatted_weights += f"{num_bits}'sd{int_value}"
                            if j < kernel_width - 1:
                                formatted_weights += ", "
                        formatted_weights += "}"
                        if i < kernel_height - 1 or d < kernel_depth - 1:
                            formatted_weights += ",\n"
                        else:
                            formatted_weights += "\n"
                formatted_weights += "};\n\n"
    return formatted_weights

# Get formatted weights as Verilog arrays with optimized bit lengths
formatted_weights = extract_and_format_weights(model)

# Print or save the weights
print(formatted_weights)  # Or save them to a file if preferred


Streaming output truncated to the last 5000 lines.
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0, 3'sd0, 3'sd0},
{3'sd0

In [ ]:
import torch
import numpy as np

class StudentModel(torch.nn.Module):
    def __init__(self, scale_factor=4):
        super(StudentModel, self).__init__()
        self.upsample = torch.nn.Sequential(
            torch.nn.Conv2d(3, 3 * (scale_factor ** 2), kernel_size=3, stride=1, padding=1),
            torch.nn.PixelShuffle(scale_factor)
        )
        self.conv1 = torch.nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1)
        self.conv2 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv3 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv4 = torch.nn.Conv2d(64, 64, kernel_size=3, stride=1, padding=1)
        self.conv5 = torch.nn.Conv2d(64, 3, kernel_size=3, stride=1, padding=1)
        self.relu = torch.nn.ReLU(inplace=True)

    def forward(self, x):
        x = self.upsample(x)
        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        x = self.conv5(x)
        x = self.relu(x)
        return x

# Load your model with weights
model = StudentModel()  # Make sure to define your actual model class
model.load_state_dict(torch.load('/content/drive/MyDrive/Projects/ULX3s-Superresolution-CNN/Efficient-Teacher-Student-Distillative-Super-Resolution-Model/ESRGAN/models/SupervisedLossOnFeatureMaps_v2.pth'))  # Load the trained model weights

def extract_and_format_weights(model):
    frac_bits = 16  # Number of fractional bits for fixed-point representation
    scale_factor = 2 ** frac_bits  # Scaling factor based on the fractional bits

    # Collect all convolution weights
    conv_weights = []
    conv_layer_indices = []

    for name, param in model.named_parameters():
        if 'conv' in name and 'weight' in name:
            weights = param.data.cpu().numpy()
            weights_scaled = np.round(weights * scale_factor).astype(np.int32)
            conv_weights.append(weights_scaled)
            conv_layer_indices.append(name.replace('.', '_'))

    # Determine the size of the 5D register
    total_convs = len(conv_weights)
    max_kernels = max(w.shape[0] for w in conv_weights)
    max_depth = max(w.shape[1] for w in conv_weights)
    max_height = max(w.shape[2] for w in conv_weights)
    max_width = max(w.shape[3] for w in conv_weights)

    formatted_weights = f"// 5D Register to store convolution weights: [ConvLayer][Kernel][Depth][Height][Width]\n"
    formatted_weights += f"// Fixed-point format: {frac_bits} fractional bits\n"
    formatted_weights += f"reg signed [31:0] conv_weights[0:{total_convs-1}][0:{max_kernels-1}][0:{max_depth-1}][0:{max_height-1}][0:{max_width-1}] = '{{\n"

    for conv_index, (weights_scaled, conv_name) in enumerate(zip(conv_weights, conv_layer_indices)):
        num_kernels, kernel_depth, kernel_height, kernel_width = weights_scaled.shape
        formatted_weights += f"// Weights for {conv_name}\n"
        formatted_weights += f"{{\n"
        for k in range(num_kernels):
            formatted_weights += f"{{\n"
            for d in range(kernel_depth):
                formatted_weights += "{"
                for i in range(kernel_height):
                    formatted_weights += "{"
                    for j in range(kernel_width):
                        int_value = weights_scaled[k, d, i, j]
                        num_bits = int(int_value).bit_length() + 1  # +1 for the sign bit
                        formatted_weights += f"{num_bits}'sd{int_value}"
                        if j < kernel_width - 1:
                            formatted_weights += ", "
                    formatted_weights += "}"
                    if i < kernel_height - 1:
                        formatted_weights += ", "
                formatted_weights += "}"
                if d < kernel_depth - 1:
                    formatted_weights += ",\n"
            formatted_weights += "}"
            if k < num_kernels - 1:
                formatted_weights += ",\n"
        formatted_weights += "}"
        if conv_index < total_convs - 1:
            formatted_weights += ",\n"
    formatted_weights += "};\n"

    return formatted_weights

# Get formatted weights as Verilog arrays with optimized bit lengths
formatted_weights = extract_and_format_weights(model)

# Print or save the weights
print(formatted_weights)  # Or save them to a file if preferred


// 5D Register to store convolution weights: [ConvLayer][Kernel][Depth][Height][Width]
// Fixed-point format: 16 fractional bits
reg signed [31:0] conv_weights[0:4][0:63][0:63][0:2][0:2] = '{
// Weights for conv1_weight
{
{
{{15'sd8400, 15'sd-12340, 13'sd3527}, {15'sd-12066, 15'sd-11606, 13'sd-2983}, {14'sd5793, 14'sd4791, 14'sd6601}},
{{14'sd-4491, 12'sd1269, 13'sd3175}, {14'sd4569, 15'sd-9277, 12'sd-1853}, {14'sd7041, 15'sd10070, 15'sd-10451}},
{{15'sd9065, 14'sd4972, 15'sd8974}, {13'sd-2049, 13'sd2351, 15'sd-9675}, {13'sd-2470, 15'sd13758, 13'sd3394}}},
{
{{14'sd6521, 14'sd6441, 12'sd1938}, {13'sd3792, 14'sd7267, 15'sd8900}, {12'sd1063, 13'sd-2078, 13'sd-3476}},
{{14'sd-6434, 14'sd-5689, 12'sd1211}, {15'sd11563, 15'sd12513, 14'sd-4893}, {15'sd13114, 15'sd10153, 13'sd-4040}},
{{15'sd-9517, 15'sd8371, 14'sd-4571}, {15'sd12415, 15'sd8479, 14'sd7376}, {14'sd5703, 15'sd-9070, 15'sd-11924}}},
{
{{15'sd11783, 15'sd-8289, 15'sd8968}, {14'sd5811, 15'sd-12238, 11'sd-922}, {14'sd7923, 14'sd-56

In [ ]:
def extract_and_format_biases(model):
    frac_bits = 16  # Number of fractional bits for fixed-point representation
    scale_factor = 2 ** frac_bits  # Scaling factor based on the fractional bits

    # Collect all biases
    biases = []
    bias_layer_indices = []

    for name, param in model.named_parameters():
        if 'conv' in name and 'bias' in name:
            bias = param.data.cpu().numpy()
            bias_scaled = np.round(bias * scale_factor).astype(np.int32)
            biases.append(bias_scaled)
            bias_layer_indices.append(name.replace('.', '_'))

    # Determine the size of the 2D register
    total_convs = len(biases)
    max_biases = max(b.shape[0] for b in biases)

    formatted_biases = f"// 2D Register to store convolution biases: [ConvLayer][Bias]\n"
    formatted_biases += f"// Fixed-point format: {frac_bits} fractional bits\n"
    formatted_biases += f"reg signed [31:0] conv_biases[0:{total_convs-1}][0:{max_biases-1}] = '{{\n"

    for conv_index, (bias_scaled, bias_name) in enumerate(zip(biases, bias_layer_indices)):
        num_biases = bias_scaled.shape[0]
        formatted_biases += f"// Biases for {bias_name}\n"
        formatted_biases += f"{{\n"
        for i in range(num_biases):
            int_value = bias_scaled[i]
            num_bits = int(int_value).bit_length() + 1  # +1 for the sign bit
            formatted_biases += f"  {num_bits}'sd{int_value}"
            if i < num_biases - 1:
                formatted_biases += ",\n"
        formatted_biases += "\n}"
        if conv_index < total_convs - 1:
            formatted_biases += ",\n"
    formatted_biases += "};\n"

    return formatted_biases

In [ ]:
biases = extract_and_format_biases(model)

print(biases)

// 2D Register to store convolution biases: [ConvLayer][Bias]
// Fixed-point format: 16 fractional bits
reg signed [31:0] conv_biases[0:4][0:63] = '{
// Biases for conv1_bias
{
  14'sd7150,
  14'sd7533,
  15'sd11206,
  15'sd-11033,
  14'sd-6885,
  12'sd2030,
  13'sd-3330,
  15'sd-8258,
  14'sd7971,
  15'sd-9152,
  12'sd1472,
  15'sd10162,
  13'sd-2538,
  15'sd-9351,
  14'sd-4750,
  13'sd3784,
  14'sd-7951,
  15'sd-11688,
  15'sd-11214,
  14'sd-4997,
  14'sd-5132,
  15'sd-10462,
  15'sd-9103,
  14'sd-5330,
  15'sd-10393,
  14'sd6746,
  1'sd0,
  13'sd2248,
  13'sd-3264,
  14'sd7197,
  15'sd-9150,
  14'sd5167,
  14'sd-7213,
  14'sd6323,
  12'sd1093,
  14'sd-4811,
  14'sd7308,
  15'sd-8254,
  14'sd-6724,
  15'sd9698,
  15'sd-8239,
  12'sd-1831,
  14'sd-4773,
  12'sd-1583,
  15'sd10951,
  14'sd-4633,
  14'sd6288,
  14'sd-6338,
  15'sd9657,
  13'sd3758,
  14'sd-5005,
  15'sd9179,
  9'sd-213,
  14'sd-6278,
  14'sd-6562,
  10'sd307,
  14'sd-4219,
  12'sd-1221,
  15'sd10295,
  12'sd-1400,
  15'